## 本番用スクリプト
(一応)ユーザーインタフェースです。

In [ ]:
# ライブラリ読み込み
import numpy as np
import matplotlib.pyplot as plt
import librosa
import json
from pydub import AudioSegment
from IPython.display import Audio
%matplotlib inline

from typing import List, Union
from modules.functions import *
from modules.network_interface import *

# セットアップ
sampling_rate = 50000
ja_sounds = load_audios('J', sampling_rate)
en_sounds = load_audios('E', sampling_rate)
submitted_answers = []

print("Done")

---
##### ここまで済ませておく
---

In [ ]:
# 試合情報の取得
match = await get_match()
print(match)

In [ ]:
# 問題情報の取得
problem = await get_problem()
print(problem)

In [ ]:
# 取得する分割データ数
N = 3
    
# 取得する分割データの指定
chunks = await get_chunks(N)
print(chunks)

# wavの取得
file_path = "./problems/"
chunks = (await get_chunk(N))["chunks"]
for chunk in chunks:
    await get_file(chunk, file_path)

In [ ]:
# 分割データ連結(分割データを最大量取るならchunks.pop(), そうでなければchunks.pop(0))
seg = to_AudioSegment(file_path + chunks.pop(0))[100:]
if N > 1:
    for chunk in chunks:
        seg += to_AudioSegment(file_path + chunk)[0:]
    
file_name = "./problems/tmp.wav"
seg.export(file_name, format="wav")
origin = load_audio(file_name, sampling_rate)

## 音源再生
# Audio(file_name, rate=sampling_rate)

In [ ]:
# 畳み込み(英語も含むか決める)
precision = [get_precision(origin, sound)
             for sound in ja_sounds + en_sounds]

In [ ]:
# ソート
answers = []
for i in np.argsort(precision)[::-1][:5]:
    print((i % 44) + 1, precision[i])
    answers.append((i % 44) + 1)

answers = sorted(list(set(answers)))
print(answers)
print([number_to_kana(n) for n in answers])

In [ ]:
# 提出
formatted_answer = list(map(lambda n: f"{n:02d}", answers))
res = await answer(problem["id"], formatted_answer)

print(formatted_answer)
print(res)

submitted_answers.append(res["answers"])
print(submitted)